In [20]:
from collections import namedtuple, defaultdict
import codecs
from config import config
from bs4 import BeautifulSoup
import pdb
import torch
from nltk.tokenize import word_tokenize
#import tokenizer
import numpy as np
import re
import pickle
import random
##Added by Richard Sun
import en_core_web_sm
nlp = en_core_web_sm.load()
from data_reader import *

In [39]:
TRAIN_DATA_PATH = "data/2014/Restaurants_Train_v2.xml"
TEST_DATA_PATH = "data/2014/Restaurants_Test_Gold.xml"
TRAIN_DATA_PATH = "data/Indonesian/indo_tweets.csv"

# TRAIN_DATA_PATH = "./data/2014/Laptop_Train_v2.xml"
# TEST_DATA_PATH = "./data/2014/Laptops_Test_Gold.xml"

GLOVE_FILE = "../data/word_embeddings/indo_vectors.txt"
OUT_FILE = config.embed_path
DATA_FILE = config.data_path
DIC_FILE = config.dic_path

In [40]:
#!pip install xlrd
import pandas as pd
#help(pd.read_excel)
data = pd.read_csv(TRAIN_DATA_PATH)

In [56]:
data.Sentiment.describe()

count        7132
unique          3
top       Neutral
freq         3091
Name: Sentiment, dtype: object

In [59]:
data[pd.isnull(data[['Sentiment']]).any(axis=1)]

,Unnamed: 0,Username,Keyword,Tweet,Sentiment,Overal Sentiment,Unnamed: 5
6130,6130,ZUL_Hasan,ISIS,TIPS jadi saksi nikah : Jangan baper .. Sesama...,NaN,NaN,remarks: the ISIS in this context does not ref...
6534,6534,tifsembiring,ISIS,"ISIS ...? ada yg bilang ""Istri Sholihah Idaman...",NaN,NaN,remarks: ISIS here does not refer to the Islam...
6538,6538,tifsembiring,HTI,"@rayestu kmbalikan pd hti nurani mas, kita kom...",NaN,NaN,"remarks: hti nurani = hati nurani, does not re..."


In [32]:
data.to_csv('data/Indonesian/indo_tweets.csv')

In [4]:
dr = data_reader(config)
dr.load_data(config.train_path)
dr_valid = data_reader(config, False)
dr_valid.load_data(config.valid_path)
dr_test = data_reader(config, False)
dr_test.load_data(config.test_path)

In [9]:
samples = dr.generate_sample(dr.data_batch)

In [15]:
words = ['ss', 'ff','df','gf']
word_freq_pair = Counter(words)

In [6]:
data_batch = dh.to_batches(data)

defaultdict(<class 'int'>, {2: 805, 0: 2164, 1: 633})


In [8]:
data2 = dh.read(TEST_DATA_PATH, False)

Dataset number: 606
Target error meal
Local Embeddings Saved!


In [60]:
sample = reader.generate_sample(train_batch)
sent_vecs, mask_vecs, label_list, sent_lens = elmo_transform(sample)

In [62]:
sent_vecs, mask_vecs, label_list, sent_lens = elmo_transform([test_batch[0]])

In [97]:
# write to pkl
with open(DATA_FILE, "wb") as f:
    pickle.dump([train_batch, test_batch],f)

In [98]:
with codecs.open(DIC_FILE, 'wb') as f:
    pickle.dump(reader.id2word, f)

In [99]:
reader.gen_vectors_glove()

1411 unk out of 5138 vocab


In [54]:
from model_att import *
model = attTSA(config)

/Users/richardsun/anaconda3/envs/allennlp/lib/python3.6/site-packages/torch/nn/modules/rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [107]:
doc = nlp(u"To be completely fair, the only redeeming factor was the food, which was above average, but couldn't make up for all the other deficiencies of Teodora.")
target = 'food'
for token in doc:
#     if 'food' in token.head.text:
#         print(token)
#     children = [t.text for t in token.children]    
#     if 'food' in children:
#         print(token)
    print('Text:'+ token.text, 'Head:'+token.head.text, 'Children:',
           [child for child in token.children])
#     if token.text == 'food':
#         a = token
#     if token.text == 'deficiencies':
#         b= token
    

Text:To Head:be Children: []
Text:be Head:was Children: [To, fair]
Text:completely Head:fair Children: []
Text:fair Head:be Children: [completely]
Text:, Head:was Children: []
Text:the Head:factor Children: []
Text:only Head:factor Children: []
Text:redeeming Head:factor Children: []
Text:factor Head:was Children: [the, only, redeeming]
Text:was Head:was Children: [be, ,, factor, food, ,, but, make, .]
Text:the Head:food Children: []
Text:food Head:was Children: [the, ,, was]
Text:, Head:food Children: []
Text:which Head:was Children: []
Text:was Head:food Children: [which, above]
Text:above Head:was Children: [average]
Text:average Head:above Children: []
Text:, Head:was Children: []
Text:but Head:was Children: []
Text:could Head:make Children: []
Text:n't Head:make Children: []
Text:make Head:was Children: [could, n't, up, for]
Text:up Head:make Children: []
Text:for Head:make Children: [deficiencies]
Text:all Head:deficiencies Children: []
Text:the Head:deficiencies Children: []
Tex

In [103]:
for item in doc.sents:
    print(item)
    print('*******')

To be completely fair, the only redeeming factor was the food, which was above average, but couldn't make up for all the other deficiencies of Teodora.
*******


In [46]:
doc = nlp(u'the screen is good but the battery sucks')
for token in doc:
#     if 'food' in token.head.text:
#         print(token)
#     children = [t.text for t in token.children]    
#     if 'food' in children:
#         print(token)
    print('Text:'+ token.text, 'Head:'+token.head.text, 'Children:',
           [child for child in token.children])

Text:the Head:screen Children: []
Text:screen Head:is Children: [the]
Text:is Head:is Children: [screen, good, but, sucks]
Text:good Head:is Children: []
Text:but Head:is Children: []
Text:the Head:battery Children: []
Text:battery Head:sucks Children: [the]
Text:sucks Head:is Children: [battery]


In [44]:
from spacy import displacy
displacy.render(doc, style='dep', jupyter=True)
# trees = doc.print_tree()
# trees[0]

In [1]:
from Layer import SimpleCat
from data_reader_general import dataHelper, data_generator
#from model_glove import *
from model_crf import *
from config import config
cat_layer = SimpleCat(config)

In [2]:
#cat_layer.load_vector()

In [189]:
sent = 'To heal the hearts of believers and to enrage kuffar, \
munafikeen and murtadeen, we present the last video memoir of Shaheed Eisa Fazili. May Allah accept him into highest Jannah'
targets = ['Eisa Fazili', 'kuffar', 'munafikeen', 'murtadeen', 'Allah']

In [164]:
sent = "But the staff was so good to us"
targets = ['staff']

In [190]:
sentence_list = []
opinion_list = []
SentInst = namedtuple("SentenceInstance", "id text text_ids text_inds opinions")
OpinionInst = namedtuple("OpinionInstance", "target_text polarity class_ind target_mask target_ids")
for target in targets:
    opinion_inst = OpinionInst(target, 'negative', None, None, None)
    opinion_list.append(opinion_inst)
sent_Inst = SentInst(1, sent, None, None, opinion_list)
sentence_list.append(sent_Inst)

In [191]:
dh = dataHelper(config)
data, words = dh.process_raw_data(sentence_list)

In [192]:
data_batch = dh.to_batches(data)

defaultdict(<class 'int'>, {2: 5})


In [193]:
dg = data_generator(config, data_batch)

In [194]:
sent_vecs, mask_vecs, label_list, sent_lens = dg.elmo_transform(data_batch)

In [195]:
#model = AspectSent(config)

In [196]:
model = torch.load('data/models/model.pt')
sent_vecs = cat_layer(sent_vecs, mask_vecs)

/Users/richardsun/anaconda3/envs/allennlp/lib/python3.6/site-packages/torch/serialization.py:367: SourceChangeWarning: source code of class 'torch.nn.modules.rnn.LSTM' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/Users/richardsun/anaconda3/envs/allennlp/lib/python3.6/site-packages/torch/serialization.py:367: SourceChangeWarning: source code of class 'torch.nn.modules.linear.Linear' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/Users/richardsun/anaconda3/envs/allennlp/lib/python3.6/site-packages/torch/serialization.py:367: SourceChangeWarning: source code of class 'torch.nn.modules.loss.CrossEntropyLoss' has changed. you

In [197]:
pred_label, best_seq = model.predict(sent_vecs, mask_vecs, sent_lens) 

In [198]:
pred_label

tensor([ 0,  0,  0,  0,  0])

In [199]:
scores, s_probs, best_seqs = model.compute_predict_scores(sent_vecs, mask_vecs, sent_lens)

In [200]:
scores

tensor([[ 2.6032, -4.7932,  0.9438],
        [ 2.6839, -5.2023,  1.0347],
        [ 2.7342, -5.2895,  1.0273],
        [ 2.7248, -5.2792,  1.0532],
        [ 2.7875, -5.3021,  1.0744]])

In [201]:
import torch.nn.functional as F
F.softmax(scores, dim=1)

tensor([[ 0.8397,  0.0005,  0.1598],
        [ 0.8385,  0.0003,  0.1612],
        [ 0.8462,  0.0003,  0.1535],
        [ 0.8415,  0.0003,  0.1582],
        [ 0.8470,  0.0003,  0.1527]])

In [203]:
from data_reader import data_reader
dr = data_reader(config)
dr.load_data(config.train_path)

In [205]:
dr.data_len

3001

In [210]:
token_list, mask_list, label_list, _ = zip(*dr.data_batch)

In [212]:
unique, counts = np.unique(label_list, return_counts=True)

In [213]:
counts

array([1816,  531,  654])

In [214]:
unique

array([0, 1, 2])

In [1]:
import torch

In [2]:
a = [[0,1,1,0],[0,1,0], [0,0,0,1,1]]


In [3]:
b = torch.zeros(3, 5)

In [5]:
for i in range(3):
    temp = a[i]
    b[i, :len(temp)] = torch.FloatTensor(temp)

In [36]:
a=torch.LongTensor([2,5,7,3,4])

In [37]:
sent_lens, perm_idx = a.sort(0, descending=True)

In [41]:
a[perm_idx[0].item()]

tensor(7)

In [38]:
sent_lens

tensor([ 7,  5,  4,  3,  2])

In [1]:
from __future__ import division
from model_att_glove import *
from data_reader_general import *
from config import config
import pickle
from Layer import GloveMaskCat
import numpy as np
import codecs
import copy
import os, sys

In [2]:
dr = data_reader(config)
train_data = dr.load_data(config.data_path+'Restaurants_Train_v2.xml.pkl')
test_data = dr.load_data(config.data_path+'Restaurants_Test_Gold.xml.pkl')
valid_data = dr.load_data(config.data_path+'valid_restaurant.pkl')
dg_train = data_generator(config, train_data)
dg_valid = data_generator(config, valid_data, False)
dg_test =data_generator(config, test_data, False)

In [3]:
cat_layer = GloveMaskCat(config)
cat_layer.load_vector()

Loaded from data/2014/vocab/local_emb.pkl with shape (4334, 300)


In [4]:
sent_vecs, mask_vecs, label_list, sent_lens, tokens = next(dg_valid.get_ids_samples())

In [5]:
mask_vecs[0]

tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0])

In [6]:
sent_vecs[0]

tensor([   52,  2175,    11,  2176,   220,   102,    10,     3,   134,
          983,  2177,     3,   451,  2178,     1,  2179,    11,   152,
          834,   233,   218,    44,   260,    32,  1482,    37,     4,
          511,    32,  2180,  2181,    38,     6,   116,  1184,    13,
            1,   141,   107,     4,   984,     2])

In [7]:
tokens[0]

['an',
 'oasis',
 'of',
 'refinement',
 ':',
 ' ',
 'food',
 ',',
 'though',
 'somewhat',
 'uneven',
 ',',
 'often',
 'reaches',
 'the',
 'pinnacles',
 'of',
 'new',
 'american',
 'fine',
 'cuisine',
 '  ',
 'chef',
 "'s",
 'passion',
 '(',
 'and',
 'kitchen',
 "'s",
 'precise',
 'execution',
 ')',
 'is',
 'most',
 'evident',
 'in',
 'the',
 'fish',
 'dishes',
 'and',
 'soups',
 '.']

In [10]:
def visualize_attention(dr_test, model):
    print("Evaluting")
    dr_test.reset_samples()
    model.eval()
    all_counter = 0
    correct_count = 0
    while dr_test.index < dr_test.data_len:
        sent, mask, label, sent_len, tokens = next(dr_test.get_ids_samples())
        sent, target = cat_layer(sent, mask)
        if config.if_gpu: 
            sent, target = sent.cuda(), target.cuda()
            label, sent_len = label.cuda(), sent_len.cuda()
        pred_label, attentions  = model.predict(sent, target, sent_len) 
        print(tokens[:3])
        print(mask[:3])
        print(attentions[:3])
        break

In [11]:
model_file = config.model_path+'model.pt'
if os.path.exists(model_file):
    model = torch.load(model_file)
    visualize_attention(dg_valid, model)

Evaluting
[['an', 'oasis', 'of', 'refinement', ':', ' ', 'food', ',', 'though', 'somewhat', 'uneven', ',', 'often', 'reaches', 'the', 'pinnacles', 'of', 'new', 'american', 'fine', 'cuisine', '  ', 'chef', "'s", 'passion', '(', 'and', 'kitchen', "'s", 'precise', 'execution', ')', 'is', 'most', 'evident', 'in', 'the', 'fish', 'dishes', 'and', 'soups', '.'], ['the', 'food', 'can', 'get', 'pricey', 'but', 'the', 'prixe', 'fixe', 'tasting', 'menu', 'is', 'the', 'greatest', 'food', 'for', 'a', 'good', 'price', 'and', 'they', 'cater', 'the', 'food', 'to', 'any', 'food', 'allergies', 'or', 'food', 'you', 'do', "n't", 'like', '.'], ['the', 'red', 'curry', 'is', 'weak', 'and', 'tasteless', ',', 'the', 'pad', 'thai', 'is', 'stuck', 'together', 'and', 'lumpy', ',', 'the', 'rice', 'is', 'often', 'overcooked', ',', 'and', 'the', 'seafood', 'is', 'pretty', 'sketchy', '.']]
tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,